In [21]:
import pandas as pd
import re

In [22]:
lamudi = pd.read_csv("/home/mario/Proyectos_Mario/proyecto_alonso/datos_extraidos/casas_lamudi.csv")

In [23]:
# Eliminar filas duplicadas
lamudi = lamudi.drop_duplicates()

# Eliminar valores nulos
lamudi = lamudi.dropna()
lamudi = lamudi.reset_index(drop="first")

In [24]:
# Se eliminan todas las casas que tienen un remate bancario o hipotecario
# ya que no representan el valor real 
casas_remate = list(lamudi[lamudi.description.str.contains("remate", case=False)].index)
lamudi = lamudi.drop(index=casas_remate)
lamudi = lamudi.reset_index(drop="first")

### Creando la columna zip_code

In [25]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace(", Oaxaca, Oaxaca", ""))
lamudi["address"] = lamudi["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("fraccionamiento", ""))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("fracc", ""))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("unidad hab", "unidad habitacional"))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("agencia de policia de", "")\
                                                        .replace("exhacienda", ""))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("res alamos", "residencial alamos"))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace("hacienda de antequera", "antequera"))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace(
    "lazaro cardenas 1ra secc", "lazaro cardenas 1a secc"))
lamudi["address"] = lamudi["address"].apply(lambda x: x.replace(
    "ejido guadalupe victoria 2da secc", "ejido guadalupe victoria segunda seccion"))



In [26]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates()

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [27]:
# Creo una lista con los datos de zip code para cada propiedad
cp_column = []
for i, d in zip(lamudi.index, lamudi.address):
    regex = re.compile(fr"(.{{0,30}}{d}.{{0,30}})", re.IGNORECASE)
    location_name = list(filter(lambda x: regex.findall(x), cp_dict.keys()))

    try:
        cp_column.append(cp_dict[location_name[0]])
    except:
        cp_column.append(None)

In [28]:
# Creo la columna zip_code
lamudi["zip_code"] = cp_column

In [29]:
# Se eliminan los valores de que se generaron en la columna zipcode
lamudi = lamudi.dropna()
lamudi = lamudi.reset_index(drop="first")

### Creando la columna parking

In [30]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,10}estacionamientos?.{0,30}|.{0,10}cocheras?.{0,30}|.{0,10}garages?.{0,30}|.{0,10}parkings?.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [31]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        elif "para cuatro" in texto.lower():
            return 4
        elif "para cinco" in texto.lower():
            return 5
        else:
            return 1

In [32]:
# Aplicar las funciones para buscar los valores de parking
lamudi["parking"] = lamudi["description"].apply(econtrar_textos)
lamudi["parking"] = lamudi["parking"].apply(asignar_valor_estacionamiento)

In [33]:
# # Se tienen 30 casas cuya descripción 
# # no incluye el número de estacionamientos
# len(lamudi[lamudi["parking"].isnull()])

In [34]:
# Se tienen algunos valores nulos en la columna zip_code
lamudi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             73 non-null     object 
 1   seller            73 non-null     object 
 2   property_type     73 non-null     object 
 3   address           73 non-null     object 
 4   price             73 non-null     int64  
 5   bedrooms          73 non-null     float64
 6   bathrooms         73 non-null     float64
 7   built_area        73 non-null     float64
 8   land_area         73 non-null     float64
 9   description       73 non-null     object 
 10  publication_date  73 non-null     object 
 11  zip_code          73 non-null     float64
 12  parking           43 non-null     float64
dtypes: float64(6), int64(1), object(6)
memory usage: 7.5+ KB


In [35]:
# Se eliminan los valores nulos
lamudi = lamudi.drop_duplicates()
lamudi = lamudi.dropna(ignore_index=True)

In [36]:
lamudi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             43 non-null     object 
 1   seller            43 non-null     object 
 2   property_type     43 non-null     object 
 3   address           43 non-null     object 
 4   price             43 non-null     int64  
 5   bedrooms          43 non-null     float64
 6   bathrooms         43 non-null     float64
 7   built_area        43 non-null     float64
 8   land_area         43 non-null     float64
 9   description       43 non-null     object 
 10  publication_date  43 non-null     object 
 11  zip_code          43 non-null     float64
 12  parking           43 non-null     float64
dtypes: float64(6), int64(1), object(6)
memory usage: 4.5+ KB


In [25]:
lamudi.to_csv("datos_lamudi.csv", index=False)

In [14]:
# # Se calculan los percentiles en los que se encuentran las distintas cantidades
# # de lugares de estacionamiento basados en el tamaño del terreno (land_area)
# # Con esto se pueden asignar valores promedio a las casas que no presentan el dato
# # de los lugares de estacionamiento
# percentiles_area = lamudi[lamudi["parking"].notnull()].land_area.quantile([0.25, 0.75, 0.95, 1])
# percentiles_parking = lamudi[lamudi["parking"].notnull()].parking.quantile([0.25, 0.75, 0.95, 1])
# print(f"Cantidad de lugares de estacionamiento por percentil\n{percentiles_parking}", end="\n\n")
# print(f"Metros cuadrados por percentil\n{percentiles_area}")

In [15]:
# # Se define una función para asignar el valor promedio de los lugares de estacionamiento
# # con base en los valores calculados mediante percentiles
# def estacionamientos_promedio(metros):
#     if metros <= 175:
#         return 1
#     elif metros <= 278:
#         return 2
#     elif metros <= 713:
#         return 3
#     else:
#         return 4

In [16]:
# # Hago una lista con los índices de las filas con valores nulos en parking
# no_parking = list(lamudi[lamudi["parking"].isnull()].index)
# # Asigno los valores de parking con base en los valores de land_area 
# # para cada una de las filas
# lamudi.loc[no_parking, "parking"] = lamudi.loc[no_parking, "parking"].apply(estacionamientos_promedio)